In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "tx_mean"

In [4]:
#Variable 
variable85="rcp85_"+var+"_monthly"
variable45="rcp45_"+var+"_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tx_mean_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tx_mean_monthly.nc']

In [5]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 12 dataframes

In [7]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"] -273.15,2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"] -273.15,2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"] -273.15,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)         
    df45f.columns = [month + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF 

In [8]:
df45list


[                      january_hist_rcp45_p10  january_t2050_rcp45_p10  \
 lat       lon                                                           
 40.041039 -89.045212                   -2.50                     0.07   
           -88.961884                   -2.50                     0.07   
           -88.878548                   -2.50                     0.06   
           -88.795219                   -2.56                     0.00   
           -88.711891                   -2.54                     0.01   
 ...                                      ...                      ...   
 66.623306 -61.629639                  -23.02                   -18.73   
           -61.546310                  -23.15                   -18.87   
           -61.462978                  -23.00                   -18.71   
           -61.379650                  -22.92                   -18.63   
           -61.296322                  -22.52                   -18.26   
 
                       january_t2080

In [9]:
def merge_multiple_dfs(df_list, on):
    df_result = df_list[0]
    for df in df_list[1:]:
        df_result = df_result.merge(df, on=on)
    return df_result

In [10]:
mkey = ["lat", "lon"]

In [11]:
dfse45 = merge_multiple_dfs(df45list[0:12], on=mkey)

In [12]:
dfse452 = dfse45.reset_index()

In [13]:
dfse452.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [14]:
dfse453 = dfse452.drop(columns=["january_hist_rcp45_p10","january_hist_rcp45_p50", "january_hist_rcp45_p90", 
                               "february_hist_rcp45_p10", "february_hist_rcp45_p50", "february_hist_rcp45_p90", 
                               "march_hist_rcp45_p10","march_hist_rcp45_p50", "march_hist_rcp45_p90", 
                               "april_hist_rcp45_p10","april_hist_rcp45_p50", "april_hist_rcp45_p90",
                              "may_hist_rcp45_p10","may_hist_rcp45_p50", "may_hist_rcp45_p90", 
                               "june_hist_rcp45_p10", "june_hist_rcp45_p50", "june_hist_rcp45_p90", 
                               "july_hist_rcp45_p10","july_hist_rcp45_p50", "july_hist_rcp45_p90", 
                               "august_hist_rcp45_p10","august_hist_rcp45_p50", "august_hist_rcp45_p90",
                              "september_hist_rcp45_p10","september_hist_rcp45_p50", "september_hist_rcp45_p90", 
                               "october_hist_rcp45_p10", "october_hist_rcp45_p50", "october_hist_rcp45_p90", 
                               "november_hist_rcp45_p10","november_hist_rcp45_p50", "november_hist_rcp45_p90", 
                               "december_hist_rcp45_p10","december_hist_rcp45_p50", "december_hist_rcp45_p90"])
dfse453


,lat,lon,january_t2050_rcp45_p10,january_t2080_rcp45_p10,january_t2050_rcp45_p50,january_t2080_rcp45_p50,january_t2050_rcp45_p90,january_t2080_rcp45_p90,february_t2050_rcp45_p10,february_t2080_rcp45_p10,...,november_t2050_rcp45_p50,november_t2080_rcp45_p50,november_t2050_rcp45_p90,november_t2080_rcp45_p90,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,0.07,-0.16,3.14,3.66,6.42,7.17,2.98,3.65,...,13.35,14.02,16.20,16.79,2.83,2.81,5.69,6.01,9.09,9.66
1,40.041039,-88.961884,0.07,-0.15,3.13,3.66,6.40,7.15,2.95,3.64,...,13.33,14.00,16.17,16.76,2.84,2.80,5.68,6.00,9.06,9.63
2,40.041039,-88.878548,0.06,-0.16,3.12,3.64,6.37,7.13,2.91,3.63,...,13.32,13.98,16.14,16.73,2.83,2.78,5.65,5.97,9.02,9.59
3,40.041039,-88.795219,0.00,-0.21,3.05,3.57,6.26,7.03,2.79,3.52,...,13.23,13.88,16.05,16.63,2.77,2.72,5.56,5.88,8.92,9.49
4,40.041039,-88.711891,0.01,-0.19,3.04,3.57,6.24,7.01,2.78,3.54,...,13.23,13.88,16.05,16.64,2.78,2.73,5.55,5.88,8.90,9.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-18.73,-17.38,-14.54,-13.43,-10.24,-9.42,-21.48,-20.19,...,-6.54,-5.66,-2.98,-2.09,-14.00,-12.48,-9.74,-8.55,-6.45,-5.26
76202,66.623306,-61.546310,-18.87,-17.54,-14.70,-13.57,-10.42,-9.58,-21.69,-20.39,...,-6.91,-6.05,-3.41,-2.53,-14.22,-12.70,-9.97,-8.79,-6.70,-5.53
76203,66.623306,-61.462978,-18.71,-17.39,-14.54,-13.40,-10.28,-9.43,-21.50,-20.21,...,-6.67,-5.82,-3.21,-2.33,-14.03,-12.52,-9.78,-8.59,-6.53,-5.37
76204,66.623306,-61.379650,-18.63,-17.32,-14.49,-13.32,-10.24,-9.38,-21.42,-20.13,...,-6.59,-5.76,-3.18,-2.32,-13.95,-12.44,-9.71,-8.52,-6.49,-5.33


In [15]:
lisdfmoh45 = []
for month in monthlist: 
    dfmoh = dfse452[["lat", "lon", month+"_hist_rcp45_p50"]]
    print (month)
    lisdfmoh45.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [16]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 12 dataframes

In [17]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"] -273.15,2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"] -273.15,2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"] -273.15,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)
    df85f.columns = [month + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF added to the list
Dataset number:  5
Percentiles Obteined
Converted to DF
S

In [18]:
df85list


[                      january_hist_rcp85_p10  january_t2050_rcp85_p10  \
 lat       lon                                                           
 40.041039 -89.045212                   -2.39                     0.61   
           -88.961884                   -2.39                     0.61   
           -88.878548                   -2.40                     0.61   
           -88.795219                   -2.46                     0.55   
           -88.711891                   -2.45                     0.55   
 ...                                      ...                      ...   
 66.623306 -61.629639                  -23.34                   -17.19   
           -61.546310                  -23.47                   -17.32   
           -61.462978                  -23.32                   -17.15   
           -61.379650                  -23.23                   -17.05   
           -61.296322                  -22.82                   -16.67   
 
                       january_t2080

In [19]:
dfse85 = merge_multiple_dfs(df85list[0:12], on=mkey).reset_index()

In [20]:
dfse85

,lat,lon,january_hist_rcp85_p10,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_hist_rcp85_p50,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_hist_rcp85_p90,january_t2050_rcp85_p90,...,november_t2080_rcp85_p90,december_hist_rcp85_p10,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_hist_rcp85_p50,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_hist_rcp85_p90,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,-2.39,0.61,2.54,1.16,3.95,5.96,4.14,7.14,...,19.10,0.55,3.65,4.95,3.99,6.62,8.02,7.07,10.14,12.14
1,40.041039,-88.961884,-2.39,0.61,2.54,1.15,3.94,5.95,4.12,7.12,...,19.06,0.55,3.63,4.95,3.97,6.60,8.01,7.04,10.11,12.11
2,40.041039,-88.878548,-2.40,0.61,2.52,1.13,3.93,5.92,4.10,7.09,...,19.03,0.54,3.62,4.95,3.95,6.57,7.98,7.01,10.07,12.08
3,40.041039,-88.795219,-2.46,0.55,2.45,1.05,3.86,5.84,4.00,7.00,...,18.93,0.47,3.55,4.88,3.87,6.49,7.90,6.91,9.97,12.00
4,40.041039,-88.711891,-2.45,0.55,2.45,1.05,3.86,5.83,3.99,6.97,...,18.93,0.48,3.56,4.90,3.87,6.48,7.90,6.90,9.96,11.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-23.34,-17.19,-12.01,-19.50,-12.67,-7.69,-15.39,-8.12,...,-0.02,-19.16,-11.53,-8.62,-15.07,-7.81,-4.92,-11.54,-3.87,0.89
76202,66.623306,-61.546310,-23.47,-17.32,-12.15,-19.64,-12.82,-7.84,-15.55,-8.29,...,-0.47,-19.35,-11.74,-8.86,-15.29,-8.05,-5.19,-11.76,-4.14,0.58
76203,66.623306,-61.462978,-23.32,-17.15,-11.99,-19.48,-12.67,-7.67,-15.39,-8.13,...,-0.27,-19.17,-11.55,-8.68,-15.11,-7.86,-5.03,-11.57,-3.99,0.70
76204,66.623306,-61.379650,-23.23,-17.05,-11.92,-19.41,-12.60,-7.61,-15.33,-8.07,...,-0.26,-19.07,-11.47,-8.62,-15.04,-7.80,-4.99,-11.49,-3.98,0.69


In [21]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [22]:
dfse852 = dfse85.drop(columns=["january_hist_rcp85_p10","january_hist_rcp85_p50", "january_hist_rcp85_p90", 
                               "february_hist_rcp85_p10", "february_hist_rcp85_p50", "february_hist_rcp85_p90", 
                               "march_hist_rcp85_p10","march_hist_rcp85_p50", "march_hist_rcp85_p90", 
                               "april_hist_rcp85_p10","april_hist_rcp85_p50", "april_hist_rcp85_p90",
                              "may_hist_rcp85_p10","may_hist_rcp85_p50", "may_hist_rcp85_p90", 
                               "june_hist_rcp85_p10", "june_hist_rcp85_p50", "june_hist_rcp85_p90", 
                               "july_hist_rcp85_p10","july_hist_rcp85_p50", "july_hist_rcp85_p90", 
                               "august_hist_rcp85_p10","august_hist_rcp85_p50", "august_hist_rcp85_p90",
                              "september_hist_rcp85_p10","september_hist_rcp85_p50", "september_hist_rcp85_p90", 
                               "october_hist_rcp85_p10", "october_hist_rcp85_p50", "october_hist_rcp85_p90", 
                               "november_hist_rcp85_p10","november_hist_rcp85_p50", "november_hist_rcp85_p90", 
                               "december_hist_rcp85_p10","december_hist_rcp85_p50", "december_hist_rcp85_p90"])
dfse852


,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2050_rcp85_p50,november_t2080_rcp85_p50,november_t2050_rcp85_p90,november_t2080_rcp85_p90,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,0.61,2.54,3.95,5.96,7.14,9.78,3.15,5.39,...,14.15,16.43,16.88,19.10,3.65,4.95,6.62,8.02,10.14,12.14
1,40.041039,-88.961884,0.61,2.54,3.94,5.95,7.12,9.76,3.14,5.39,...,14.12,16.40,16.85,19.06,3.63,4.95,6.60,8.01,10.11,12.11
2,40.041039,-88.878548,0.61,2.52,3.93,5.92,7.09,9.72,3.11,5.37,...,14.10,16.37,16.81,19.03,3.62,4.95,6.57,7.98,10.07,12.08
3,40.041039,-88.795219,0.55,2.45,3.86,5.84,7.00,9.63,3.00,5.27,...,14.01,16.28,16.71,18.93,3.55,4.88,6.49,7.90,9.97,12.00
4,40.041039,-88.711891,0.55,2.45,3.86,5.83,6.97,9.60,3.00,5.28,...,14.02,16.28,16.71,18.93,3.56,4.90,6.48,7.90,9.96,11.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-17.19,-12.01,-12.67,-7.69,-8.12,-2.22,-19.79,-15.45,...,-5.58,-3.94,-2.30,-0.02,-11.53,-8.62,-7.81,-4.92,-3.87,0.89
76202,66.623306,-61.546310,-17.32,-12.15,-12.82,-7.84,-8.29,-2.43,-20.00,-15.67,...,-5.97,-4.36,-2.74,-0.47,-11.74,-8.86,-8.05,-5.19,-4.14,0.58
76203,66.623306,-61.462978,-17.15,-11.99,-12.67,-7.67,-8.13,-2.33,-19.81,-15.47,...,-5.75,-4.15,-2.54,-0.27,-11.55,-8.68,-7.86,-5.03,-3.99,0.70
76204,66.623306,-61.379650,-17.05,-11.92,-12.60,-7.61,-8.07,-2.33,-19.72,-15.39,...,-5.69,-4.11,-2.52,-0.26,-11.47,-8.62,-7.80,-4.99,-3.98,0.69


In [23]:
lisdfmoh85 = []
for month in monthlist: 
    dfmoh = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    print (month)
    lisdfmoh85.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


In [24]:
lisdfmoh85

[             lat        lon  january_hist_rcp85_p50
 0      40.041039 -89.045212                    1.16
 1      40.041039 -88.961884                    1.15
 2      40.041039 -88.878548                    1.13
 3      40.041039 -88.795219                    1.05
 4      40.041039 -88.711891                    1.05
 ...          ...        ...                     ...
 76201  66.623306 -61.629639                  -19.50
 76202  66.623306 -61.546310                  -19.64
 76203  66.623306 -61.462978                  -19.48
 76204  66.623306 -61.379650                  -19.41
 76205  66.623306 -61.296322                  -19.03
 
 [76206 rows x 3 columns],              lat        lon  february_hist_rcp85_p50
 0      40.041039 -89.045212                     4.58
 1      40.041039 -88.961884                     4.58
 2      40.041039 -88.878548                     4.56
 3      40.041039 -88.795219                     4.45
 4      40.041039 -88.711891                     4.45
 ...        

# Merge df45 and 85

# Extract hist

## Getting historic mean for each month

In [25]:
### Function
def mean_month(month, dfse45, dfse85):
    print(month)
    dfw45 = dfse45[["lat", "lon", month+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[month+"_hist_p50"] = round((dfsew[month+"_hist_rcp45_p50"]+ dfsew[month+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", month+"_hist_p50"]]
    return (dftgwh3)


### All months historic mean

In [26]:
listdfmo = []
i=0
for month in monthlist:
    print(month)
    dfm = mean_month(month, lisdfmoh45[i], lisdfmoh85[i])
    i = i+1
    listdfmo.append(dfm)

january
january
february
february
march
march
april
april
may
may
june
june
july
july
august
august
september
september
october
october
november
november
december
december


# Merge means with the 45 and 85 DF

In [27]:
dftgseall = dfse852.merge(dfse452, on=mkey)
i=0
for df in listdfmo:
    dftgseall2 = dftgseall.merge(listdfmo[i], on=mkey)
    i = i +1


In [28]:
dftgseall

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,0.61,2.54,3.95,5.96,7.14,9.78,3.15,5.39,...,16.79,0.67,2.83,2.81,4.03,5.69,6.01,7.31,9.09,9.66
1,40.041039,-88.961884,0.61,2.54,3.94,5.95,7.12,9.76,3.14,5.39,...,16.76,0.67,2.84,2.80,4.01,5.68,6.00,7.28,9.06,9.63
2,40.041039,-88.878548,0.61,2.52,3.93,5.92,7.09,9.72,3.11,5.37,...,16.73,0.66,2.83,2.78,3.99,5.65,5.97,7.25,9.02,9.59
3,40.041039,-88.795219,0.55,2.45,3.86,5.84,7.00,9.63,3.00,5.27,...,16.63,0.59,2.77,2.72,3.91,5.56,5.88,7.16,8.92,9.49
4,40.041039,-88.711891,0.55,2.45,3.86,5.83,6.97,9.60,3.00,5.28,...,16.64,0.60,2.78,2.73,3.91,5.55,5.88,7.15,8.90,9.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-17.19,-12.01,-12.67,-7.69,-8.12,-2.22,-19.79,-15.45,...,-2.09,-19.04,-14.00,-12.48,-14.90,-9.74,-8.55,-11.56,-6.45,-5.26
76202,66.623306,-61.546310,-17.32,-12.15,-12.82,-7.84,-8.29,-2.43,-20.00,-15.67,...,-2.53,-19.24,-14.22,-12.70,-15.12,-9.97,-8.79,-11.79,-6.70,-5.53
76203,66.623306,-61.462978,-17.15,-11.99,-12.67,-7.67,-8.13,-2.33,-19.81,-15.47,...,-2.33,-19.06,-14.03,-12.52,-14.92,-9.78,-8.59,-11.60,-6.53,-5.37
76204,66.623306,-61.379650,-17.05,-11.92,-12.60,-7.61,-8.07,-2.33,-19.72,-15.39,...,-2.32,-18.97,-13.95,-12.44,-14.85,-9.71,-8.52,-11.53,-6.49,-5.33


In [29]:
dfmop50 = merge_multiple_dfs(listdfmo[0:12], on=mkey)

In [30]:
dfmop50

,lat,lon,january_hist_p50,february_hist_p50,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,1.20,4.57,10.74,18.10,24.10,28.98,30.74,29.80,26.34,19.40,11.60,4.01
1,40.041039,-88.961884,1.19,4.56,10.70,18.07,24.08,28.98,30.74,29.80,26.35,19.38,11.57,3.99
2,40.041039,-88.878548,1.17,4.54,10.66,18.02,24.06,28.98,30.73,29.81,26.38,19.36,11.55,3.97
3,40.041039,-88.795219,1.10,4.44,10.51,17.92,24.00,28.95,30.66,29.74,26.33,19.27,11.46,3.89
4,40.041039,-88.711891,1.09,4.43,10.50,17.91,24.03,29.02,30.70,29.80,26.40,19.28,11.46,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-19.47,-21.14,-18.07,-11.30,-2.42,4.70,9.25,8.26,2.30,-3.54,-9.48,-14.98
76202,66.623306,-61.546310,-19.61,-21.35,-18.41,-11.74,-2.86,4.29,8.84,7.84,1.82,-3.98,-9.85,-15.20
76203,66.623306,-61.462978,-19.45,-21.16,-18.15,-11.43,-2.57,4.56,9.09,8.12,2.14,-3.68,-9.59,-15.02
76204,66.623306,-61.379650,-19.38,-21.08,-18.05,-11.32,-2.47,4.64,9.18,8.22,2.26,-3.58,-9.50,-14.94


In [31]:
dftgseall2 = dfse852.merge(dfse452, on=mkey)

In [32]:
#dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [33]:
dftgseall2

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,0.61,2.54,3.95,5.96,7.14,9.78,3.15,5.39,...,16.79,0.67,2.83,2.81,4.03,5.69,6.01,7.31,9.09,9.66
1,40.041039,-88.961884,0.61,2.54,3.94,5.95,7.12,9.76,3.14,5.39,...,16.76,0.67,2.84,2.80,4.01,5.68,6.00,7.28,9.06,9.63
2,40.041039,-88.878548,0.61,2.52,3.93,5.92,7.09,9.72,3.11,5.37,...,16.73,0.66,2.83,2.78,3.99,5.65,5.97,7.25,9.02,9.59
3,40.041039,-88.795219,0.55,2.45,3.86,5.84,7.00,9.63,3.00,5.27,...,16.63,0.59,2.77,2.72,3.91,5.56,5.88,7.16,8.92,9.49
4,40.041039,-88.711891,0.55,2.45,3.86,5.83,6.97,9.60,3.00,5.28,...,16.64,0.60,2.78,2.73,3.91,5.55,5.88,7.15,8.90,9.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-17.19,-12.01,-12.67,-7.69,-8.12,-2.22,-19.79,-15.45,...,-2.09,-19.04,-14.00,-12.48,-14.90,-9.74,-8.55,-11.56,-6.45,-5.26
76202,66.623306,-61.546310,-17.32,-12.15,-12.82,-7.84,-8.29,-2.43,-20.00,-15.67,...,-2.53,-19.24,-14.22,-12.70,-15.12,-9.97,-8.79,-11.79,-6.70,-5.53
76203,66.623306,-61.462978,-17.15,-11.99,-12.67,-7.67,-8.13,-2.33,-19.81,-15.47,...,-2.33,-19.06,-14.03,-12.52,-14.92,-9.78,-8.59,-11.60,-6.53,-5.37
76204,66.623306,-61.379650,-17.05,-11.92,-12.60,-7.61,-8.07,-2.33,-19.72,-15.39,...,-2.32,-18.97,-13.95,-12.44,-14.85,-9.71,-8.52,-11.53,-6.49,-5.33


In [34]:
dftgseall3 = dftgseall2.merge(dfmop50, on=mkey)

In [35]:
dftgseall3.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_monthly.feather")

In [36]:
dftgseall3

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,0.61,2.54,3.95,5.96,7.14,9.78,3.15,5.39,...,10.74,18.10,24.10,28.98,30.74,29.80,26.34,19.40,11.60,4.01
1,40.041039,-88.961884,0.61,2.54,3.94,5.95,7.12,9.76,3.14,5.39,...,10.70,18.07,24.08,28.98,30.74,29.80,26.35,19.38,11.57,3.99
2,40.041039,-88.878548,0.61,2.52,3.93,5.92,7.09,9.72,3.11,5.37,...,10.66,18.02,24.06,28.98,30.73,29.81,26.38,19.36,11.55,3.97
3,40.041039,-88.795219,0.55,2.45,3.86,5.84,7.00,9.63,3.00,5.27,...,10.51,17.92,24.00,28.95,30.66,29.74,26.33,19.27,11.46,3.89
4,40.041039,-88.711891,0.55,2.45,3.86,5.83,6.97,9.60,3.00,5.28,...,10.50,17.91,24.03,29.02,30.70,29.80,26.40,19.28,11.46,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-17.19,-12.01,-12.67,-7.69,-8.12,-2.22,-19.79,-15.45,...,-18.07,-11.30,-2.42,4.70,9.25,8.26,2.30,-3.54,-9.48,-14.98
76202,66.623306,-61.546310,-17.32,-12.15,-12.82,-7.84,-8.29,-2.43,-20.00,-15.67,...,-18.41,-11.74,-2.86,4.29,8.84,7.84,1.82,-3.98,-9.85,-15.20
76203,66.623306,-61.462978,-17.15,-11.99,-12.67,-7.67,-8.13,-2.33,-19.81,-15.47,...,-18.15,-11.43,-2.57,4.56,9.09,8.12,2.14,-3.68,-9.59,-15.02
76204,66.623306,-61.379650,-17.05,-11.92,-12.60,-7.61,-8.07,-2.33,-19.72,-15.39,...,-18.05,-11.32,-2.47,4.64,9.18,8.22,2.26,-3.58,-9.50,-14.94


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [37]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [38]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [39]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [40]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Estrie-Monteregie-Laval-Montreal_tx_mean_monthly.json: No such file or directory
ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Outaouais_tx_mean_monthly.json: No such file or directory


OUTAOUAIS


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Laurentides_tx_mean_monthly.json: No such file or directory


...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_tx_mean_monthly.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
